In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langchain.agents import AgentExecutor, Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_structured_chat_agent

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [2]:
driver_service = Service(ChromeDriverManager().install())
driver_service.start() 
# Use Service().start() to keep chromedriver running
# Process independence, LangChain still survives after releasing variables

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(model="gpt-4o",temperature=0.1)

In [5]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [6]:
def google_chrome_search(kw: str)-> str:
    import time
    import random
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    print('test the function is invoked')
    #setup chrome options
    options = Options()

    # Disguise the user agent (make Google think it's a normal user)
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

    # Open Chrome in Incognito mode
    options.add_argument("--incognito")

    # Disable Automation Flag
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Maximize Window
    options.add_argument("--start-maximized")

    # # start Chrome 
    # service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=driver_service, options=options)
    driver.get("https://www.google.com")

    # wait page loading
    time.sleep(random.uniform(1, 3))

    # Find the search box
    search_box = driver.find_element(By.NAME, "q")

    # Simulate mouse movement to the search box
    action = ActionChains(driver)
    action.move_to_element(search_box).perform()

    # Enter keywords
    search_box.send_keys(kw)

    time.sleep(random.uniform(1, 2))

    # Press enter to search
    search_box.send_keys(Keys.RETURN)

    return None

In [7]:
# google_chrome_search(kw="machine learning")

In [8]:
def google_chrome_search_tool(kw: str)-> str:
    google_chrome_search(kw=kw)

    return "Agentfinished, you just need to response the webpage is already open."



In [9]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    ),
    Tool(
        name="GoogleChromeSearch",
        func=google_chrome_search_tool,
        description="Use this tool to search some web pages through google if asking you search something online"
    )
]

In [10]:
system_message=SystemMessagePromptTemplate.from_template(
    """ Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}

        ** And if you invoked any tool, return the tool's return content with your own words. Do not more iterations.**
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
     
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [11]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [12]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_7665/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [15]:
response = agent_executor.invoke({
    "input": "What is 15^2?",
    "history": memory.chat_memory.messages,  
    # "agent_scratchpad": ''
})

print(response)



> Entering new AgentExecutor chain...
```json
{
  "action": "Final Answer",
  "action_input": "15^2 is equal to 225."
}
```

> Finished chain.
{'input': 'What is 15^2?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15^2 is equal to 225.', additional_kwargs={}, response_metadata={})], 'output': '15^2 is equal to 225.'}


In [16]:
response = agent_executor.invoke({
    "input": "What is the time now?",
    "history": memory.chat_memory.messages,  
})

print(response)



> Entering new AgentExecutor chain...
Thought: I need to provide the current local time in a JSON blob format.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 16:29:52```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 16:29:52."
}
```

> Finished chain.
{'input': 'What is the time now?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15^2 is equal to 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 16:29:52.', additional_kwargs={}, response_metadata={})], 'output': 'The current local time is 16:29:52.'}


In [17]:
response = agent_executor.invoke({
    "input": "Could you help me to open the web page for the current NBA trade news?",
    "history": memory.chat_memory.messages,  
})

print(response['output'])



> Entering new AgentExecutor chain...
Thought: To find the current NBA trade news, I should perform a web search.
Action:
```json
{
  "action": "GoogleChromeSearch",
  "action_input": "current NBA trade news"
}
```test the function is invoked
Agentfinished, you just need to response the webpage is already open.Action:
```json
{
  "action": "Final Answer",
  "action_input": "The webpage for the current NBA trade news is already open."
}
```

> Finished chain.
The webpage for the current NBA trade news is already open.


In [18]:
# driver_service.stop() 